In [3]:
import pandas as pd
import pm4py
import numpy as np
import json
from collections import OrderedDict

In [4]:
#로그 기록이 담긴 csv를 Heuristic Model로 변환
df_example_log = pd.read_csv('AnonymizedEventData.csv')


event_log_pm4py = pm4py.format_dataframe(df_example_log, case_id='TicketNum', activity_key='Status',
                                         timestamp_key='Time')
heu_model = pm4py.discover_heuristics_net(event_log_pm4py)

<ipython-input-4-0ed870acd1c0>:5: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  event_log_pm4py = pm4py.format_dataframe(df_example_log, case_id='TicketNum', activity_key='Status',
c:\Users\hb990\pythontemp\anaconda3\lib\site-packages\pm4py\utils.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[constants.CASE_CONCEPT_NAME] = df[constants.CASE_CONCEPT_NAME].astype("string")
c:\Users\hb990\pythontemp\anaconda3\lib\site-packages\pm4py\utils.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [5]:
print(heu_model.dependency_matrix)

{'Acknowledged': {'Closed': 0.5}, 'Assignment': {'Manually Acknowledged': 0.996415770609319, 'acknowledged notification': -0.24374262875114663, 'analysis/research/tech note': 0.3877683799609629, 'communication with customer': -0.21794871794871795, 'communication with provider': -0.5, 'pending confirmation': 0.2807017543859649, 'pending customer': 0.055323590814196244, 'pending provider': 0.0, 'pending release': 0.43478260869565216, 'pending repair': 0.014814814814814815, 'pending vendor': -0.10714285714285714, 'reassigned-addl work required': 0.5307459677419355, 'reassigned-misrouted': 0.34512325830653806, 'restored to service': 0.9958677685950413, 'status request': -0.6875, 'waiting parts': 0.25}, 'Closed': {'Closed': 0.9995114802149487, 'Open': 0.9991896272285251, 'communication with customer': -0.65625, 'communication with vendor': -0.2222222222222222, 'pending repair': -0.3472222222222222, 'pending vendor': -0.6746987951807228, 'status request': -0.24390243902439024}, 'DEADLINE ALE

In [11]:
#Heurisic Model을 3d 그래프 input에 적합한 json으로 변환
file_data = OrderedDict()

#Nodes
nodes = []

nodes.append({"id":"start", "nodelabel": "start", "value": 1})

for a,b in heu_model.activities_occurrences.items():
  nodes.append({"id": a, "nodelabel":a , "value": int(b)})

nodes.append({"id":"end", "nodelabel": "end", "value": 1})
file_data["nodes"] = nodes

#Links
links = []

for f in heu_model.start_activities:
  for g, h in f.items():
    links.append({"source":"start", "target": g, "weight": 1})

for a in heu_model.performance_matrix.keys():
  for b in heu_model.performance_matrix[a].keys():
    links.append({"source":a, "target": b, "weight": heu_model.performance_matrix[a][b]})

for c in heu_model.end_activities:
  for d, e in c.items():
    links.append({"source":d, "target": "end", "weight": 1})

file_data["links"]=links

In [15]:
#json으로 출력
print(json.dumps(file_data, ensure_ascii=False, indent="\t"))

with open('test1.json', 'w', encoding="utf-8") as make_file:
  json.dump(file_data, make_file, ensure_ascii=False, indent="\t")

{
	"nodes": [
		{
			"id": "start",
			"nodelabel": "start",
			"value": 1
		},
		{
			"id": "analysis/research/tech note",
			"nodelabel": "analysis/research/tech note",
			"value": 19615
		},
		{
			"id": "acknowledged notification",
			"nodelabel": "acknowledged notification",
			"value": 16069
		},
		{
			"id": "Closed",
			"nodelabel": "Closed",
			"value": 15767
		},
		{
			"id": "Open",
			"nodelabel": "Open",
			"value": 13561
		},
		{
			"id": "Assignment",
			"nodelabel": "Assignment",
			"value": 11625
		},
		{
			"id": "pending customer",
			"nodelabel": "pending customer",
			"value": 9353
		},
		{
			"id": "restored to service",
			"nodelabel": "restored to service",
			"value": 5980
		},
		{
			"id": "alert stage 1",
			"nodelabel": "alert stage 1",
			"value": 5531
		},
		{
			"id": "reassigned-addl work required",
			"nodelabel": "reassigned-addl work required",
			"value": 3875
		},
		{
			"id": "alert stage 2",
			"nodelabel": "alert stage 2",
			"value": 3018
		},
	